In [13]:
import collections
import numpy as np
import pegasus as pg
import pandas as pd
import pegasusio as io
import matplotlib.pyplot as plt
# %cd -q ..

In [14]:
file_names = ["../human-other-kidney2.h5ad"]

In [15]:
# Three functions for percent expression, one for finding intersections of lists
def filter_adata(adata, filters):
    adata_flt = adata.copy()
    adata_flt.obs["passed_qc"] = False 
    adata_flt.obs.loc[np.logical_or.reduce(filters), "passed_qc"] = True
    pg.filter_data(adata_flt)
    return adata_flt

def calculate_expression(adata, gene, threshold):
    cind = adata.var.index.get_loc(gene)
    expr = adata.X[:, cind].toarray().flatten()
    colname = gene + "_" + str(threshold)
    adata.obs.loc[expr > 0, colname] = gene + '+'
    adata.obs.loc[expr <= 0, colname] = gene + '-'
    return adata

def expression_cluster_table(adata, exp_col, gene_name, cleaned_list):
    cell_type, gene_pos, gene_neg, gene_total, gene_pct  = [], [], [], [], []

    for grp in sorted(cleaned_list):
        grp_adata = filter_adata(adata, [adata.obs.cell_type == grp])
#         cl_adata = filter_adata(grp_adata, [grp_adata.obs.cluster == cl])
        cnt = collections.Counter(grp_adata.obs[exp_col])
        cpos = cnt[gene_name + "+"]
        cneg = cnt[gene_name + "-"]
        if cpos + cneg == 0:
            print(grp, cl)
            return
        pct = round(cpos / (cpos + cneg) * 100, 3) 
        cell_type.append(grp)
        gene_pos.append(cpos)
        gene_neg.append(cneg)
        gene_total.append(cpos+cneg)
        gene_pct.append(pct)
    df_dict = {"cell_type": cell_type, 
               gene_name + "+": gene_pos, gene_name + "-": gene_neg, gene_name + "_Total": gene_total, gene_name + "%": gene_pct}
    exp_df = pd.DataFrame(data=df_dict)
    return exp_df

In [16]:
def intersection(list1, list2):
    return [entry for entry in list1 if entry in list2]

data_paths = [
    ["Other/", ["human-other-brain", "human-other-colon_epi", "human-other-colon_fib", "human-other-colon_imm", 
                "human-other-kidney2", "human-other-liver", "human-other-Olfactory_Epithelium", "human-other-retina", 
                "human-other-skin"]]
]

In [17]:
def expression_and_plots(human_or_mouse, data_paths, gene_percent_list, gene_feature_list):
    for data_path in data_paths:
        datas = data_path[1]
        for file_name in datas:
            print("CURRENT FILE: " + file_name)
            # reading data
            data = pg.read_input(human_or_mouse + "/" + data_path[0] + "/" + file_name + '/' + file_name + '.h5ad')
            # need to normalize for everything to work well
            pg.identify_robust_genes(data)
            pg.log_norm(data)
#             number_cells = len(data)

            # List of cell types for use in the percent expression functions
            cleanedList = [x for x in list(set(data.obs.cell_type)) if str(x) != 'nan'] 
            cleanedList = sorted(cleanedList)

            # defining the genes we want to look at. First array is for feature plots and dot plots, second is for the gene percents.
            

#         FOR MOUSE: the genes are all in lowercase, so we are switching it here.
#             

            # creating dataframe
            df = pd.DataFrame(cleanedList, columns=["cell_type"])
            firstOne = False
            # percent annotation
            for gene in gene_percent_list:
                try:
                    print("CURRENTLY ATTEMPTING GENE: " + gene)
                    data_gene = calculate_expression(data, gene, 0)
                    df_gene = expression_cluster_table(data_gene, gene+"_0", gene, cleanedList)
        #             display(df_gene)
                    name_and_T = gene + "_Total"
                    name_and_sign = gene + "%"
                    if firstOne == False:
                        df["# cells"] = df_gene[name_and_T].values
                        firstOne = True

                    df[name_and_sign] = df_gene[name_and_sign].values
                    print("GENE " + gene + " % EXPRESSION HAS BEEN INPUTTED INTO THE DATAFRAME")

                except KeyError:
                    print("THE GENE " + gene + " IS NOT PRESENT IN THIS DATASET.")
                print()
                print()
            df.to_csv('Mouse/TabulaSenis30m/' + file_name + '/' + file_name + "_%expression.csv")
            full_gene_list = []
            for gene in data.var.index:
                full_gene_list.append(gene)

            # if a gene listed in feature list isn't actually in the data, it gets removed here
            update_feature_list = intersection(full_gene_list, gene_feature_list)
            update_feature_list = [var for x in gene_feature_list for var in update_feature_list if var == x]

            #adding umap coordinates
            data.obs['umap1'] = pd.to_numeric(data.obs['umap1'], errors='coerce')
            data.obs['umap2'] = pd.to_numeric(data.obs['umap2'], errors='coerce')
            umap = data.obs[["umap1", "umap2"]]
            data.obsm['X_umap'] = umap.values

            # creating feature plots
            express_list = ['cell_type']
            count = 0
            for gene in update_feature_list:
                express_list.append(gene)
                if len(express_list) == 6:
                    plt.figure()
                    pg.scatter(data, attrs=express_list, ncols=3)
                    plt.savefig("Mouse/TabulaSenis30m/" + file_name + "/" + file_name + "_feature" + str(count) + ".png", dpi=300)
                    express_list = ['cell_type']
                    count = count + 1
            print(len(update_feature_list))
            if len(update_feature_list) != 0:
                plt.figure()
                pg.dotplot(data, genes=update_feature_list, groupby="cell_type")
                plt.savefig("Mouse/TabulaSenis30m/" + file_name + "/" + file_name + "_test.png", dpi=300, bbox_inches='tight')

            print("COMPLETED FILE: " + file_name)
            print()
            print()

            
            
# For humans, all genes should be in caps.
# For mouse, the first character of the gene should be capitalized, everything else lowercase.
gene_feature_list = ["CDKN2A", "HAVCR1", "SEMA4A", "HAVCR2", "TIMD4", "TIGIT", "LAG3", "PDCD1", "IL27", "PRDM1", "CMAF", "IL10", "TGFB", "IL1R", "IL1B", "IL6", "TNFA", "TBX21", "PTPRC", "EPCAM", "PECAM1", "CD3D", "CD3G", "CD3E", "C1QA", "CD79A", "CD79B", "LYVE1"]
gene_percent_list = ["CDKN2A", "HAVCR1", "SEMA4A", "HAVCR2", "TIMD4"]
